### Android 的两种崩溃
- Java 崩溃：Java 崩溃就是在 Java 代码中，出现了未捕获异常，导致程序异常退出
- Native 崩溃：一般都是因为在 Native 代码中访问非法地址，也可能是地址对齐出现了问题，或者发生了程序主动 abort，这些都会产生相应的 signal 信号，导致程序异常退出。

---

#### Native 崩溃的捕获流程
- 编译端。编译 C/C++ 代码时，需要将带符号信息的文件保留下来
- 客户端。捕获到崩溃时候，将收集到尽可能多的有用信息写入日志文件，然后选择合适的时机上传到服务器
- 服务端。读取客户端上报的日志文件，寻找适合的符号文件，生成可读的 C/C++ 调用栈

![image](crash1_page1.png)

---

#### Native 崩溃捕获的难点
- 想达到 99%，在各种恶劣条件下依然可靠，后面付出的努力会远远高于前期
- 最核心的是怎么样保证客户端在各种极端情况下依然可以生成崩溃日志。在崩溃时，程序会处于一个不安全的状态，如果处理不当，非常容易发送二次崩溃
- 生成崩溃日志时会有哪些比较棘手的情况呢？
  - 文件句柄泄漏，导致创建日志文件失败，怎么办？
    - 应对方式：我们需要提前申请文件句柄 fd 预留，防止出现这种情况
  - 因为栈溢出了，导致日志生成失败，怎么办？
    - 应对方式：为了防止栈溢出导致进程没有空间创建调用栈执行处理函数，通常会使用常见的 signalstack。在一些特殊情况，我们可能还需要直接替换当前栈，所以这里也需要在堆中预留部分空间。
  - 整个堆的内存都耗尽了，导致日志生成失败，怎么办？
    - 应对方式：这个时候我们无法安全地分配内存，也不敢使用 stl 或者 libc 的函数，因为它们内部实现会分配堆内存。Breakpad 做的比较彻底，重新封装了 Linux Syscall Support，来避免直接调用 libc
  - 堆破坏或二次崩溃导致日志生成失败，怎么办？
    - 应对方式：Breakpad 会从原进程 fork 出子进程去收集崩溃现场，此外涉及与 Java 相关的，一般也会用子进程去操作。在一些特殊的情况，我们还可能需要从子进程 fork 出孙进程。

---

#### 选择合适的崩溃服务
- 腾讯的 Bugly
- 阿里的 啄木鸟平台
- 网易云捕
- Google 的 Firebase

- 从产品化跟社区维护来说，Bugly 在国内做的最好
- 从技术深度跟捕获能力来说，阿里 UC 浏览器内核团队打造的啄木鸟平台最佳

---

### 如何客观地衡量崩溃
- 应用 PV 崩溃率
- 启动崩溃率
- 重复崩溃率

```
UV 崩溃率 = 发生崩溃的 UV / 登录 UV
```

---

### 如何客观地衡量稳定性
- 怎么去发现应用中的 ANR 异常呢？
  - 使用 FileObserver 监听 /data/anr/traces.txt 的变化
    - 很多高版本的 ROM，已经没有读取这个文件的权限了
  - 监控消息队列的运行时间
    - 这个方案无法准确地判断是否真正出现了 ANR 异常，也无法得到完整的 ANR 日志
    - 更应该放到卡顿的性能范畴

- 有哪些应用退出的情形
  - 主动自杀：Process.killProcess()、exit() 等
  - 崩溃：出现了 Java 或 Native 崩溃
  - 系统重启；系统出现异常、断电、用户主动重启等，我们可以通过比较应用开机运行时间是否比之前记录的值更小
  - 被系统杀死：被 low memory killer 杀掉、从系统的任务管理器中划掉等。
  - ANR

> 可以在应用启动的时候设定一个标志，在主动自杀或崩溃后更新标志，这样下次启动时通过检测这个标志就能确认运行期间是否发生过异常退出。对应上面的五种退出场景，我们排除掉主动自杀和崩溃（崩溃会单独的统计）这两种场景，希望可以监控到剩下三种的异常退出，理论上这个异常捕获机制是可以达到 100% 覆盖的。

```
UV 异常率 = 发生异常退出或崩溃的 UV / 登录 UV
```

---

### 课后作业
- [Breakpad](https://chromium.googlesource.com/breakpad/breakpad/+/master) 是一个跨平台的开源项目，今天的课后作业是使用 Breakpad 来捕获一个Native 崩溃，并在留言区写下你今天学习和练习后的总结与思考。

---

### 参考资料
- GitHub 资料 [group](https://github.com/AndroidAdvanceWithGeektime)
- [Sample](https://github.com/AndroidAdvanceWithGeektime/Chapter01)

---